In [1]:
import os
import random
random.seed(42)
import time
import numpy as np
np.random.seed(42)
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
import bcolz
import pandas as pd
import zarr
import allel; print('scikit-allel', allel.__version__)
import scipy

# check which version is installed
print(allel.__version__)


scikit-allel 1.3.2
1.3.2


In [2]:
### Load def
%load_ext Cython

In [4]:
%%cython

import numpy as np
cimport numpy as np
import cython

def pdiff_int32(np.int32_t[:] x):
    cdef:
        np.int32_t[:] d
        Py_ssize_t i, j, k, n, n_pairs
    n = x.shape[0]
    n_pairs = (n * (n - 1)) // 2
    d = np.empty(n_pairs, dtype='i4')
    k = 0
    for i in range(n):
        for j in range(i+1, n):
            d[k] = x[j] - x[i]
            k += 1
    return np.asarray(d)

In [5]:
def snp_ascertainment(gt, pos, chrom, start, stop, min_maf):
    
    # SNP ascertainment
    ac = gt.count_alleles(max_allele=3)
    af = ac.to_frequencies()
    loc_asc = (ac.max_allele() == 1) & (af[:, :2].min(axis=1) > min_maf)
    loc_region = np.zeros(pos.size, dtype='b1')
    loc_region[pos.locate_range(start, stop)] = True
    loc_asc &= loc_region
    # log('SNP ascertainment', chrom, start, stop, pop, nnz(loc_asc))
    print('SNP ascertainment', chrom, start, stop, pop)
    
    # extract genotypes for population
#    popidx = tbl_samples.eq('population', pop).values('index').list()
#    gt = gt.subset(variants=loc_asc)
    gt = gt.compress(loc_asc, axis=0)
    gn = gt.to_n_alt()
    
    return pos[loc_asc], gn

In [6]:
# collect LD
def collect_ld_distance(pos, gn, n_rep, window_size, window_step=1, permute=False):
    assert pos.shape[0] == gn.shape[0]
    dist = None
    r2 = None
    # calculate expected number of data points
    expectedlen = ((window_size * (window_size - 1)) // 2) * n_rep
    for i in range(n_rep):
        # pick a random index
        start_index = np.random.randint(0, pos.shape[0]-window_size)
        stop_index = start_index + window_size
        posr = pos[start_index:stop_index:window_step]
#         log(i, start_index, pos[start_index], pos[stop_index])
        gnr = gn[start_index:stop_index:window_step]
        if permute:
            for i in range(window_size//window_step):
                gnr[i] = np.roll(gnr[i], i)
        x = pdiff_int32(posr)
        y = allel.rogers_huff_r(gnr)**2
        if dist is None:
            dist = bcolz.carray(x, expectedlen=expectedlen)
            r2 = bcolz.carray(y, expectedlen=expectedlen)
        else:
            dist.append(x)
            r2.append(y)
    return dist[:], r2[:]


In [7]:
def ld_dist(gt, pos, region, pop, min_maf, n_rep, window_size, window_step, permute):
    
    _, chrom, start, stop = region
    
    # get genotypes
    print(min_maf)
    pos, gn = snp_ascertainment(gt, pos, chrom, start, stop, min_maf)
    
    # collect LD
    dist, r2 = collect_ld_distance(pos, gn, n_rep=n_rep, window_size=window_size, window_step=window_step, permute=permute)
    
    return dist, r2

def compute_ld_dist_multi(gt, pos, pop, region, window_sizes, window_steps, n_reps, min_maf):

    # accumulate data
    dist = None
    r2 = None
    min_maf = 0.1

    for window_size, window_step, n_rep in zip(window_sizes, window_steps, n_reps):
        dist_r, r2_r = ld_dist(gt, pos, region, pop, min_maf=0.1, window_size=window_size,window_step=window_step, n_rep=n_rep, permute=False)
        if dist is None:
            dist = dist_r
            r2 = r2_r
        else:
            # combine
            dist = np.concatenate([dist, dist_r])
            r2 = np.concatenate([r2, r2_r])
            
    return dist, r2


In [8]:
#sub set gt for a given pop

def selectInd(pop, gt):
    # locate samples from given population
    loc_samples_pop = panel[panel.population == pop].callset_index.values

    # create a genotype array of the selected samples
    gt_pop = gt.take(loc_samples_pop, axis=1)
    return gt_pop


In [15]:
### Start script

zarr_path = '/home/daron/bioInf/wilding/vcf_store/wilding_urbano.unifiedGenotyper.cov14x.passQC.zarr'
print(zarr_path)
callset = zarr.open_group(zarr_path, mode='r')



/home/daron/bioInf/wilding/vcf_store/wilding_urbano.unifiedGenotyper.cov14x.passQC.zarr


In [19]:
chrom = "3R"

gt_path = '/'+chrom+'/calldata/GT'
gt = allel.GenotypeChunkedArray(callset['/3R/calldata/GT'])

pos_path = '/'+chrom+'/samples'
pos = allel.SortedIndex(callset['3R/variants/POS'])

samples_path = '/'+chrom+'/samples'
samples_path = '/'+chrom+'/samples'

# load meta file from all pop
panel = pd.read_csv("/home/daron/bioInf/wilding/popstructure/statDesc/ldDecay/meta", sep='\t', usecols=['id', 'population'])

# coorespondance of samples order between snp and meta file
samples_list = list(callset[samples_path])
samples_callset_index = [samples_list.index(s) for s in panel['id']]
panel['callset_index'] = samples_callset_index


In [20]:
pop_path = "/home/daron/bioInf/wilding/vcf_store/pop"
file=open(pop_path, "r+")

total_df = pd.DataFrame()

for pop in file.readlines():
    pop = pop.rstrip()
    print(pop)
    gt_pop = selectInd(pop, gt)
    
    # Establish baseline via permutation method
    perm_baseline = dict()

    region = '3R', '3R', 1000000, 37000000
    min_maf = .1
    n_rep = 500
    window_size = 30
    window_step = 1
    permute = True

    dist, r2 = ld_dist(gt_pop, pos, region, pop, min_maf=min_maf, window_size=window_size, window_step=window_step, 
                       permute=permute, n_rep=n_rep)
    # plot_ld_decay(dist, r2, title='%s baseline' % pop)
    perm_baseline[pop] = r2
    
    ### get LD decay
    region = '3R', '3R', 1000000, 37000000
    window_sizes = (100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 51200, 102400, 204800)
#    window_sizes = (100, 200, 400)
    window_steps = tuple(x//100 for x in window_sizes)
    n_reps = (4000, 4000, 4000, 4000, 1000, 1000, 1000, 500, 500, 500, 500, 500, 500)
#    n_reps = (4000, 4000, 4000)
    min_maf = .1

    # get data
    dist, r2 = compute_ld_dist_multi(gt_pop, pos, pop, region, window_sizes, window_steps, n_reps, min_maf)
    dist = dist[np.isfinite(r2)]
    r2 = r2[np.isfinite(r2)]

    ### scaled x, y value for ploting
    xmax = 14000000
    nbin = 400

    # apply binning
    bins = np.logspace(1, np.log10(xmax), nbin)
    s, e, _ = scipy.stats.binned_statistic(dist, r2, statistic=np.nanmean, bins=bins)
    x = (e[:-1] + e[1:]) / 2

    # adjust for sampling effects
    y = s - np.nanmean(perm_baseline[pop])
    
    # create df
    d = {'distance':x,'ld':y}
    df = pd.DataFrame(d)
    df = df.assign(pop=pop, chrom=chrom)
    total_df = pd.concat([total_df, df], axis=0)





BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
0.1
SNP ascertainment 3R 1000000 37000000 BZVurb
DLAurb
0.1
SNP ascertainment 3R 1000000 37000000 DLAurb
0.1
SNP ascertainment 3R 1000000 37000000 DLAurb
0.1
SNP ascertainment 3R 1000000 37000000 DLAurb
0.1
SNP ascertainment 3R 1000000 37000000 DLAurb
0.1
SNP ascertainment 3R 1000000 37000000 DLAurb
0.1
SNP ascertainment 3R 1000000 37000000 DLAurb
0.1
SNP ascertainment 3R 1000000 37000000 DLAurb
0.1
SN

MemoryError: Unable to allocate 812. MiB for an array with shape (106418204,) and data type int64

In [25]:
### print whole dataset

total_df = total_df.dropna()
txt_fn = "/home/daron/bioInf/wilding/popstructure/statDesc/allpop.ld.txt"
total_df.to_csv(txt_fn, sep='\t', index=False,  float_format='%.8f')



         distance        ld    pop chrom
0    1.018053e+01  0.545784     KE    3R
1    1.054810e+01       NaN     KE    3R
2    1.092894e+01  0.540039     KE    3R
3    1.132353e+01       NaN     KE    3R
4    1.173236e+01       NaN     KE    3R
..            ...       ...    ...   ...
394  1.193656e+07       NaN  UGgam    3R
395  1.236753e+07       NaN  UGgam    3R
396  1.281406e+07       NaN  UGgam    3R
397  1.327671e+07       NaN  UGgam    3R
398  1.375607e+07       NaN  UGgam    3R

[798 rows x 4 columns]
